<a href="https://colab.research.google.com/github/Nov05/DS-Unit-3-Sprint-2-SQL-and-Databases/blob/master/sc/2019_08_16_sc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# created by nov05 on 2019-08-16

# Part 1 - Making and populating a Database

In [1]:
import sqlite3
import pandas as pd

# The version number of this module, as a string. 
# This is not the version of the SQLite library.
print(sqlite3.version) # 2.6.0
# The version number of the run-time SQLite library, as a string.
print(sqlite3.sqlite_version) # 3.22.0
!python -c "import sqlite3; print(sqlite3.sqlite_version)" # 3.22.0 or 3.29.0

2.6.0
3.22.0
3.22.0


https://github.com/Nov05/DS-Unit-3-Sprint-2-SQL-and-Databases/blob/master/module2-sql-for-analysis/rpg_db.sqlite3.sql  

In [91]:
%%writefile demo_data.py

import sqlite3

db_name = 'demo_data.sqlite3'
conn = sqlite3.connect(db_name)
curs = conn.cursor()

# create table and insert values
sqls = [''] * 5
sqls[0] = """
CREATE TABLE IF NOT EXISTS demo
("s" varchar(30) NOT NULL,
 "x" integer NOT NULL,
 "y" integer NOT NULL,
 PRIMARY KEY("s")
);"""
sqls[1] = "INSERT INTO demo VALUES ('g', 3, 9);"
sqls[2] = "INSERT INTO demo VALUES ('v', 5, 7);"
sqls[3] = "INSERT INTO demo VALUES ('f', 8, 7);"
sqls[4] = "COMMIT;"
for sql in sqls:
    curs.execute(sql)

sql = "SELECT COUNT(*) FROM demo"
curs.execute(sql)
print(curs.fetchall()[0][0])

sql = """
SELECT COUNT(*) FROM demo
 WHERE x >= 5 
   AND y >= 5
"""
curs.execute(sql)
print(curs.fetchall()[0][0])

sql = """
SELECT COUNT(DISTINCT(y)) FROM demo
"""
curs.execute(sql)
print(curs.fetchall()[0][0])

curs.close()
conn.close()

Writing demo_data.py


In [94]:
!python demo_data.py

3
2
2


# Part 2 - The Northwind Database

# Part 3 - Sailing the Northwind Seas

In [0]:
!wget -cO - https://github.com/Nov05/DS-Unit-3-Sprint-2-SQL-and-Databases/blob/master/sc/northwind_small.sqlite3?raw=true > northwind_small.sqlite3

In [115]:
%%writefile northwind.py

import sqlite3
import pandas as pd

db_name = 'northwind_small.sqlite3'
conn = sqlite3.connect(db_name)
curs = conn.cursor()

############################################################
print("""
  SELECT name FROM sqlite_master
   WEHRE type='table'
ORDER BY name; 
""")
print(curs.execute("SELECT name FROM sqlite_master \
WHERE type='table' \
ORDER BY name;").fetchall())

print("""
SELECT sql FROM sqlite_master
 WHERE name="Customer";
""")
print(curs.execute('SELECT sql FROM sqlite_master \
WHERE name="Customer";').fetchall()[0][0])
print()

############################################################
# start from here I am not going to capitalize the SQL 
# tokens cause it is really unnecessary for practice...
############################################################

############################################################
print("""
What are the ten most expensive items (per unit price) \
in the database?
""")
sql = """
  select ID, ProductName, UnitPrice
    from Product
order by UnitPrice desc limit 10
"""
with pd.option_context('display.max_rows', None, 
                       'display.max_columns', None):
    print(pd.read_sql(sql, conn))
print()

############################################################
print("""
What is the average age of an employee at the time of \
their hiring? (Hint: a lot of arithmetic works with \
dates.)
""")
sql = """
select ID, FirstName, LastName, 
       HireDate - BirthDate as HireAtAge
  from Employee
"""
with pd.option_context('display.max_rows', None, 
                       'display.max_columns', None):
    print(pd.read_sql(sql, conn))
print()

############################################################
print("""
(Stretch) How does the average age of employee \
at hire vary by city?
""")
sql = """
select avg(HireAtAge), City 
from (
    select HireDate - BirthDate as HireAtAge,
           City
      from Employee
)
group by City
"""
with pd.option_context('display.max_rows', None, 
                       'display.max_columns', None):
    print(pd.read_sql(sql, conn))
print()

############################################################
print("""
What are the ten most expensive items (per unit price) \
in the database and their suppliers?
""")
sql = """
   select p.ID, p.ProductName, p.UnitPrice,
          s.CompanyName as Supplier
     from Product as p
left join Supplier as s
       on p.SupplierID = s.ID
 order by p.UnitPrice desc limit 10
"""
with pd.option_context('display.max_rows', None, 
                       'display.max_columns', None):
    print(pd.read_sql(sql, conn))
print()

############################################################
print("""
What is the largest category \
(by number of unique products in it)?
Note: I could have used max(). But I think in real life \
people usually want to see a list (report).
""")
sql = """
   select Count(c.ID) as ProductTypes,
          c.ID,
          c.CategoryName
     from Category as c
left join Product as p
       on c.ID = p.CategoryID
 group by c.ID
 order by ProductTypes desc
    limit 1
"""
with pd.option_context('display.max_rows', None, 
                       'display.max_columns', None):
    print(pd.read_sql(sql, conn))
print()

############################################################
print("""
(Stretch) Who's the employee with the most territories? \
Use TerritoryId (not name, region, or other fields) \
as the unique identifier for territories.
""")
sql = """
  select t.EmployeeID,
         t.TerritoryCount,
         e.FirstName,
         e.LastName
    from (
            select EmployeeID,
                   count(TerritoryID) as TerritoryCount
              from EmployeeTerritory
          group by EmployeeID
          order by TerritoryCount desc
             limit 1
         ) as t
left join Employee as e
       on t.EmployeeID = e.ID
"""
with pd.option_context('display.max_rows', None, 
                       'display.max_columns', None):
    print(pd.read_sql(sql, conn))
print()

curs.close()
conn.close()

Overwriting northwind.py


In [116]:
!python northwind.py


  SELECT name FROM sqlite_master
   WEHRE type='table'
ORDER BY name; 

[('Category',), ('Customer',), ('CustomerCustomerDemo',), ('CustomerDemographic',), ('Employee',), ('EmployeeTerritory',), ('Order',), ('OrderDetail',), ('Product',), ('Region',), ('Shipper',), ('Supplier',), ('Territory',)]

SELECT sql FROM sqlite_master
 WHERE name="Customer";

CREATE TABLE "Customer" 
(
  "Id" VARCHAR(8000) PRIMARY KEY, 
  "CompanyName" VARCHAR(8000) NULL, 
  "ContactName" VARCHAR(8000) NULL, 
  "ContactTitle" VARCHAR(8000) NULL, 
  "Address" VARCHAR(8000) NULL, 
  "City" VARCHAR(8000) NULL, 
  "Region" VARCHAR(8000) NULL, 
  "PostalCode" VARCHAR(8000) NULL, 
  "Country" VARCHAR(8000) NULL, 
  "Phone" VARCHAR(8000) NULL, 
  "Fax" VARCHAR(8000) NULL 
)


What are the ten most expensive items (per unit price) in the database?

   Id              ProductName  UnitPrice
0  38            Côte de Blaye     263.50
1  29  Thüringer Rostbratwurst     123.79
2   9          Mishi Kobe Niku      97.00
3  2

# Part 4 - Questions (and your Answers)

Answer the following questions, baseline ~3-5 sentences each, as if they were interview screening questions (a form you fill when applying for a job):

* **In the Northwind database, what is the type of relationship between the Employee and Territory tables?**    

```
Answer: 1:n. Or one to many. One employee could be linked to multiple territories. 
```

* **What is a situation where a document store (like MongoDB) is appropriate, and what is a situation where it is not appropriate?**    

```
Answer:  
Appropriate:     
1. When the data needs to horizontally scale up;    
2. When the data is mostly document like, a.k.a. unstructured;  
3. When the business website is more about content, e.g. blogs, videos, etc.
Inappropriate:  
1. When it is a highly transactional systems;
2. When the systems are hightly coupled
```

* **What is "NewSQL", and what is it trying to achieve?**      

```
Answer: NewSQL is a new approach to relational databases that wants to combine transactional ACID (atomicity, consistency, isolation, durability) guarantees of good RDBMSs and the horizontal scalability of NoSQL. It sounds like a perfect solution, the best of both worlds.  
```

# Part 5 - Turn it in!

Add all the files you wrote (demo_data.py, northwind.py), as well as this file with your answers to part 4, to your weekly repo (DS-Unit-3-Sprint-2-SQL-and-Databases). You're also encouraged to include the output from your queries as docstring comments, to facilitate grading. Commit, push, and await feedback from your PM. Thanks for your hard work!

If you got this far, check out the larger Northwind database - your queries should run on it as well, with richer results.

In [111]:
import os
os.getcwd()

'/content'

In [0]:
from google.colab import files
files.download('demo_data.py') 
files.download('northwind.py')

https://docs.oracle.com/cd/E11882_01/server.112/e41573/perf_overview.htm#PFGRF025   
Home / Oracle Database Online Documentation Library, 11g Release 2 (11.2)  
Database Performance Tuning Guide